In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import os
import cv2
import torch
from PIL import Image
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15966, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 15966 (delta 43), reused 46 (delta 20), pack-reused 15880
Receiving objects: 100% (15966/15966), 14.70 MiB | 25.65 MiB/s, done.
Resolving deltas: 100% (10938/10938), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.3/616.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.8 MB/s eta

In [ ]:
detection_model = torch.hub.load('.', 'custom', path='/content/models/detection_model/best.pt', source='local')
recognition_model = torch.hub.load('.', 'custom', path='/content/models/recognition_model/best.pt', source='local')

YOLOv5 🚀 v7.0-217-g8c45e51 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)



[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/pyparsing-3.1.1.dist-info/METADATA'


Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 v7.0-217-g8c45e51 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 


[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/cycler-0.11.0.dist-info/METADATA'


Model summary: 157 layers, 7080247 parameters, 0 gradients, 16.0 GFLOPs
Adding AutoShape... 


In [ ]:
images = '/content/images/'

for img in os.listdir(images):
    try:
        img_path = os.path.join(images, img)
        result = detection_model(img_path)
        crops = result.crop(save=True)
    except (OSError, cv2.error) as e:
        print(f"Error processing image {img}: {e}")
        continue

Saved 1 image to runs/detect/exp
Saved results to runs/detect/exp



In [ ]:
directory = '/content/yolov5/runs/detect'

for filename in os.listdir(directory):
    for crops in os.listdir(os.path.join(directory, filename)):
        if crops == "crops":
            for plate in os.listdir(os.path.join(directory, filename, crops)):
                for img in os.listdir(os.path.join(directory, filename, crops, plate)):
                    img_path = os.path.join(directory, filename, crops, plate, img)
                    image = cv2.imread(img_path)
                    saved_img_path = '/content/saved_images'
                    # print(img)
                    cv2.imwrite('/content/saved_images' + "//" + img, image)

In [ ]:
input_folder = '/content/saved_images'
output_folder = '/content/resized_images'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
# Iterate over the files in the input folder
for filename in os.listdir(input_folder):
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)

    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Read the image
        image = cv2.imread(input_path)

        # Check if the image was successfully read
        if image is not None:
            ratio = 600 / image.shape[1]
            width = round(image.shape[1] * ratio)
            height = round(image.shape[0] * ratio)

            # Resize the image
            resized_image = cv2.resize(image, (width, height))

            # Save the resized image to the output folder
            cv2.imwrite(output_path, resized_image)
            print(f"Resized image saved: {output_path}")
        else:
            print(f"Failed to read image: {input_path}")
    else:
        print(f"Skipping non-image file: {input_path}")

Resized image saved: /content/resized_images/2.jpg


In [ ]:
model_result = {}  # Use a dictionary instead of a list

# Specify the path to your input image
image_path = '/content/resized_images/2.jpg'

# Perform object detection on the image
results = recognition_model(image_path)

# Get the predicted labels and bounding box coordinates
labels = results.xyxyn[0][:, -1].cpu().numpy()
boxes = results.xyxyn[0][:, :-1].cpu().numpy()

# Get the class names associated with the model
class_names = recognition_model.module.names if hasattr(recognition_model, 'module') else recognition_model.names

# Reverse the order of labels and bounding boxes
reversed_labels = labels[::-1]
reversed_boxes = boxes[::-1]

# Iterate over the reversed bounding box coordinates and labels
for label, box in zip(reversed_labels, reversed_boxes):
    # Get the label name using the model's class names
    label_name = class_names[int(label)]

    # Store the label name and reversed bounding box in the model_result dictionary
    model_result[label_name] = box[0]

# Print the model_result dictionary
print(model_result)

{'five': 0.39845732, 'alif': 0.70307034, 'yaa': 0.53996605, 'seen': 0.8121415, 'seven': 0.18367451, 'eight': 0.066700965, 'four': 0.29052845}


In [ ]:
lst = []
for label, box in zip(reversed_labels, reversed_boxes):
    # Get the label name using the model's class names
    label_name = class_names[int(label)]

    # Print the label name and reversed bounding box
    # print("Label: {}, Bounding Box: {}".format(label_name, box))
    # print(label_name, box[0])
    lst.append(box[0])

In [ ]:
lst.sort()

In [ ]:
# Initialize an empty list to store the matching keys
matching_keys = []

# Iterate over the elements in lst
for item in lst:
    for key, value in model_result.items():
        if value == item:
            matching_keys.append(key)
            break  # Break out of the inner loop after finding a match

# Print the matching keys
print(matching_keys)

['eight', 'seven', 'four', 'five', 'yaa', 'alif', 'seen']


In [ ]:
replace_arabic_characters_and_digits = {"zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9,
                                        "alif":"أ", "baa":"ب", "geem":"ج", "daal":"د", "raa":"ر", "seen":"س", "saad":"ص", "taa":"ط",
                                        "ein":"ع", "faa":"ف", "qaaf":"ق", "laam":"ل", "meem":"م", "noon":"ن", "haa":"ه", "waaw":"و", "yaa":"ي"}

In [ ]:
# Initialize an empty list to store the corresponding values
matching_values = []

# Iterate over the keys in matching_keys
for key in matching_keys:
    if key in replace_arabic_characters_and_digits:
        value = replace_arabic_characters_and_digits[key]
        matching_values.append(value)

# Print the matching values
print(matching_values)

[8, 7, 4, 5, 'ي', 'أ', 'س']


In [ ]:
numbers = ''
characters = ''

for item in matching_values:
    if isinstance(item, int):
        numbers += str(item)
    elif isinstance(item, str):
        characters += item
        separated_characters = list(characters[::-1])

result_list = [numbers, separated_characters]

In [ ]:
result_list

['8745', ['س', 'أ', 'ي']]

In [ ]:
replace_first_element = {"س": "الإسكندرية", "ر": "الشرقية", "د":"الدقهلية", "م":"المنوفية","ب":"البحيرة","ل":"كفر الشيخ", "ع":"الغربية", "ق":"القليوبية",
                         "ف":"الفيوم", "و":"بني سويف", "ن":"المنيا","ي":"أسيوط", "ه":"سوهاج"}

replace_second_element_if_first_geem = {"ه": "مطروح", "ب":"الوادي الجديد"}

replace_second_element_if_first_saad = {"أ": "قنا", "ق":"الأقصر", "و":"أسوان"}

replace_second_element_if_first_taa = {"س":"السويس", "ص":"الاسماعيلية", "ع":"بورسعيد", "د":"دمياط", "أ":"شمال سيناء", "ج":"جنوب سيناء", "ر":"البحر الأحمر"}

In [ ]:
first_char = result_list[1][0]
sec_char = result_list[1][1]

if len(result_list[0]) == 3 and len(result_list[1]) == 3:
    value = "القاهرة"
elif len(result_list[0]) == 4 and len(result_list[1]) == 2:
    value = "الجيزة"
else:
    if first_char == "ج":
        value = replace_second_element_if_first_geem[sec_char]
    elif first_char == "ص":
        value = replace_second_element_if_first_saad[sec_char]
    elif first_char == "ط":
        value = replace_second_element_if_first_taa[sec_char]
    else:
        value = replace_first_element[first_char]

In [ ]:
result_list.append(value)

result_list

['8745', ['س', 'أ', 'ي'], 'الإسكندرية', 'الإسكندرية']